### Create dev env

In [ ]:
!conda create -y -n nm python==3.9
!conda activate nm
!pip install -r requirements.txt

## Local inference with DeepSparse

In [1]:
from deepsparse import Pipeline
from deepsparse.image_classification.constants import IMAGENET_LABELS

cv_pipeline = Pipeline.create(
  task='image_classification', 
  model_path='zoo:cv/classification/resnet_v1-18/pytorch/sparseml/imagenet/pruned85_quant-none-vnni',
)

/opt/conda/envs/nm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
DeepSparse, Copyright 2021-present / Neuralmagic, Inc. version: 1.3.2 COMMUNITY | (7d31c4bf) (release) (optimized) (system=avx512, binary=avx512)
[7f11a3a4d740 >WARN<  operator() ./src/include/wand/utility/warnings.hpp:14] Generating emulated code for quantized (INT8) operations since no VNNI instructions were detected. Set NM_FAST_VNNI_EMULATION=1 to increase performance at the expense of accuracy.


In [2]:
input_image = "cat.jpg"
inference = cv_pipeline(images=input_image)
inference

ImageClassificationOutput(labels=[282], scores=[18.798709869384766])

In [7]:
from PIL import Image
input_image = Image.open("cat.jpg")
inference = cv_pipeline(images=[input_image])
inference

ImageClassificationOutput(labels=[282], scores=[18.798709869384766])

In [4]:
labels = [IMAGENET_LABELS[label_id] for label_id in inference.labels][0]
scores = inference.scores[0]

labels, scores

('tiger cat', 18.798709869384766)

## SageMaker deployment

In [1]:
import boto3, sagemaker, base64

region = "eu-west-1"
sm_client = boto3.client(service_name="sagemaker", region_name=region)
runtime_sm_client = boto3.client("sagemaker-runtime", region_name=region)
session = sagemaker.Session(boto_session=boto3.Session(region_name=region))
role = "YOUR ARN ROLE"
NAME = "neural-magic"

### Deploy with custom Docker container + Flask backend

In [ ]:
# build image and push to ECR
!./flask/build_and_push_to_ecr.sh

In [8]:
ecr_image = "257446244580.dkr.ecr.eu-west-1.amazonaws.com/custom-images:neuralmagic"

model = sagemaker.model.Model(image_uri=ecr_image,
                              name=NAME,
                              role=role,
                              sagemaker_session=session,
                              predictor_cls=sagemaker.Predictor
                             )

predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.c5.xlarge', 
                         endpoint_name=NAME)

-----!

In [9]:
predictor = sagemaker.Predictor(endpoint_name=NAME, sagemaker_session=session)

In [10]:
with open("cat.jpg", "rb") as img_file:
    b64_string = base64.b64encode(img_file.read())

predictor.predict('{"image": "'+b64_string.decode('utf-8')+'"}')

b'{"label": "tiger cat", "score": 18.79037857055664}'

In [11]:
for _ in range(10000): _ = predictor.predict('{"image": "'+b64_string.decode('utf-8')+'"}')

In [12]:
sm_client.delete_model(ModelName=NAME)
sm_client.delete_endpoint_config(EndpointConfigName=NAME)
sm_client.delete_endpoint(EndpointName=NAME)

{'ResponseMetadata': {'RequestId': 'dc0145c6-2a94-464d-9853-331f9f730f54',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dc0145c6-2a94-464d-9853-331f9f730f54',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 11 Feb 2023 18:40:08 GMT'},
  'RetryAttempts': 0}}

### Deploy using NeuralMagic `deepsparse.server` functionality (FastAPI backend)

In [ ]:
# build image + push to ECR + deploy to SageMaker
!python ./fastapi/endpoint.py create

In [17]:
import base64, json

# with open("cat.jpg", "rb") as f:
#     payload = f.read()

with open("cat.jpg", "rb") as img_file:
    b64_string = base64.b64encode(img_file.read())
    payload = {"request": b64_string.decode('utf-8'), "type": "image/jpeg"}
    payload = json.dumps(payload)

response = runtime_sm_client.invoke_endpoint(
    EndpointName=NAME, 
    ContentType="application/json", 
    Accept='application/json',
    Body=payload
)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (422) from primary with message "{"detail":[{"loc":["body","request"],"msg":"field required","type":"value_error.missing"}]}". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/neural-magic-test in account 257446244580 for more information.

In [18]:
sm_client.delete_model(ModelName=NAME)
sm_client.delete_endpoint_config(EndpointConfigName=NAME)
sm_client.delete_endpoint(EndpointName=NAME)

{'ResponseMetadata': {'RequestId': '839ce252-51cd-4b9d-ab4e-1388a9a17664',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '839ce252-51cd-4b9d-ab4e-1388a9a17664',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 11 Feb 2023 18:53:54 GMT'},
  'RetryAttempts': 0}}

### Random useful

`(echo -n '{"image": "'; base64 cat.jpg; echo '"}') | curl -H "Content-Type: application/json" -d @-  http://127.0.0.1:8080/invocations`

`curl -X POST http://127.0.0.1:5543/predict/from_files -H 'accept: application/json' -H 'Content-Type: multipart/form-data' -F 'request=@cat.jpg;type=image/jpeg'`

`deepsparse.server config config-local.yaml`